In [109]:
import pandas as pd
import numpy as np

import time

from ast import literal_eval
import re
import os

In [63]:
envdf = pd.read_csv("/Users/eclin/Desktop/MA Thesis/02 MA Thesis Current Project/EnvData/envdata_for_analysis.csv", converters={"NewLocs": literal_eval})


In [64]:
envdf.columns

Index(['DisNo.', 'Historic', 'Classification Key', 'Disaster Group',
       'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype',
       'External IDs', 'Event Name', 'ISO', 'Country', 'Subregion', 'Region',
       'Location', 'Origin', 'Associated Types', 'OFDA/BHA Response', 'Appeal',
       'Declaration', 'AID Contribution ('000 US$)', 'Magnitude',
       'Magnitude Scale', 'Latitude', 'Longitude', 'River Basin', 'Start Year',
       'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day',
       'Total Deaths', 'No. Injured', 'No. Affected', 'No. Homeless',
       'Total Affected', 'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damage ('000 US$)', 'Insured Damage, Adjusted ('000 US$)',
       'Total Damage ('000 US$)', 'Total Damage, Adjusted ('000 US$)', 'CPI',
       'Admin Units', 'Entry Date', 'Last Update', 'LocClean', 'LocationList',
       'NewLocs', 'start_date', 'end_date', 'start_year', 'end_year'],
      

In [71]:
state_dict = {
    # https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States#States.
    "AK": "Alaska",
    "AL": "Alabama",
    "AR": "Arkansas",
    "AZ": "Arizona",
    "CA": "California",
    "CO": "Colorado",
    "CT": "Connecticut",
    "DE": "Delaware",
    "FL": "Florida",
    "GA": "Georgia",
    "HI": "Hawaii",
    "IA": "Iowa",
    "ID": "Idaho",
    "IL": "Illinois",
    "IN": "Indiana",
    "KS": "Kansas",
    "KY": "Kentucky",
    "LA": "Louisiana",
    "MA": "Massachusetts",
    "MD": "Maryland",
    "ME": "Maine",
    "MI": "Michigan",
    "MN": "Minnesota",
    "MO": "Missouri",
    "MS": "Mississippi",
    "MT": "Montana",
    "NC": "North Carolina",
    "ND": "North Dakota",
    "NE": "Nebraska",
    "NH": "New Hampshire",
    "NJ": "New Jersey",
    "NM": "New Mexico",
    "NV": "Nevada",
    "NY": "New York",
    "OH": "Ohio",
    "OK": "Oklahoma",
    "OR": "Oregon",
    "PA": "Pennsylvania",
    "RI": "Rhode Island",
    "SC": "South Carolina",
    "SD": "South Dakota",
    "TN": "Tennessee",
    "TX": "Texas",
    "UT": "Utah",
    "VA": "Virginia",
    "VT": "Vermont",
    "WA": "Washington",
    "WI": "Wisconsin",
    "WV": "West Virginia",
    "WY": "Wyoming",
    # https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States#Federal_district.
    "DC": "District of Columbia",
    # https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States#Inhabited_territories.
    "AS": "American Samoa",
    "GU": "Guam GU",
    "MP": "Northern Mariana Islands",
    "PR": "Puerto Rico PR",
    "VI": "U.S. Virgin Islands",
}

In [215]:
envdf["FullStateList"] = ""
for i, row in envdf.iterrows():
    locs = row["NewLocs"]

    all_locs = []
    for loc in locs:
        all_locs.append(state_dict[loc])

    envdf.at[i, "FullStateList"] = all_locs

In [248]:
by_state = envdf.explode("NewLocs")

In [249]:
by_state.columns

Index(['DisNo.', 'Historic', 'Classification Key', 'Disaster Group',
       'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype',
       'External IDs', 'Event Name', 'ISO', 'Country', 'Subregion', 'Region',
       'Location', 'Origin', 'Associated Types', 'OFDA/BHA Response', 'Appeal',
       'Declaration', 'AID Contribution ('000 US$)', 'Magnitude',
       'Magnitude Scale', 'Latitude', 'Longitude', 'River Basin', 'Start Year',
       'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day',
       'Total Deaths', 'No. Injured', 'No. Affected', 'No. Homeless',
       'Total Affected', 'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damage ('000 US$)', 'Insured Damage, Adjusted ('000 US$)',
       'Total Damage ('000 US$)', 'Total Damage, Adjusted ('000 US$)', 'CPI',
       'Admin Units', 'Entry Date', 'Last Update', 'LocClean', 'LocationList',
       'NewLocs', 'start_date', 'end_date', 'start_year', 'end_year',
       

In [250]:
groupedby_state_year = pd.DataFrame(by_state.groupby(["NewLocs", "end_year"])['NewLocs'].count())
groupedby_state_year = groupedby_state_year.rename(columns={groupedby_state_year.columns[0]: "year_state_count"})
groupedby_state_year.reset_index(inplace = True)
groupedby_state_year.to_csv("/Users/eclin/Desktop/MA Thesis/02 MA Thesis Current Project/Data For Analysis/year_state_data.csv", index = False)

In [251]:
groupedby_state = pd.DataFrame(by_state.groupby(["NewLocs"])['NewLocs'].count())
groupedby_state = groupedby_state.rename(columns={groupedby_state.columns[0]: "state_count"})
groupedby_state.reset_index(inplace = True)
groupedby_state.to_csv("/Users/eclin/Desktop/MA Thesis/02 MA Thesis Current Project/Data For Analysis/state_data.csv", index = False)

In [245]:
groupedby_year = pd.DataFrame(by_state.groupby(["end_year"])['end_year'].count())
groupedby_year = groupedby_year.rename(columns={groupedby_year.columns[0]: "year_count"})
groupedby_year.reset_index(inplace = True)
groupedby_year.to_csv("/Users/eclin/Desktop/MA Thesis/02 MA Thesis Current Project/Data For Analysis/year_data.csv", index = False)

In [72]:
years = range(2008, 2025, 1)
all_year_dict = []
for year in years: 
    env_search = envdf[envdf["Start Year"] == year]
    all_subtypes = list(pd.unique(env_search["Disaster Subtype"]))

    year_dict = {"subtype" : [],
                "locations" : []}

    for subtype in all_subtypes:
        year_dict["subtype"].append(subtype)
        subtype_df = env_search[env_search["Disaster Subtype"] == subtype]
        
        all_states = []
        for lst in subtype_df["NewLocs"]:
            all_states.extend(lst)
            
        all_states = list(set(all_states))

        all_states_clean = []
        for state in all_states:
            all_states_clean.append(state_dict[state])
            
        
        year_dict["locations"].append(all_states_clean)

    year_df = pd.DataFrame(year_dict)
    year_df["year"] = year

    all_year_dict.append(year_df)

all_year_df = pd.concat(all_year_dict)
all_year_df

,subtype,locations,year
0,Riverine flood,"[Iowa, Kentucky, Minnesota, Oklahoma, Californ...",2008
1,Tornado,"[Iowa, Kentucky, Minnesota, Oklahoma, Ohio, Mi...",2008
2,Ground movement,[Nevada],2008
3,Fire (Miscellaneous),[Pennsylvania],2008
4,Forest fire,[California],2008
...,...,...,...
4,Flood (General),"[Iowa, Minnesota, Texas, Colorado, Wyoming, Lo...",2024
5,Forest fire,"[California, Washington]",2024
6,Tropical cyclone,"[Kentucky, Texas, Mississippi, Pennsylvania, G...",2024
7,Storm (General),"[Missouri, Ohio, Minnesota, Mississippi, Monta...",2024


In [73]:
all_year_df.to_excel("/Users/eclin/Desktop/MA Thesis/02 MA Thesis Current Project/EnvData/cleaned_trends_to_search.xlsx")

In [75]:
t = pd.read_csv("/Users/eclin/Desktop/MA Thesis/02 MA Thesis Current Project/EnvData/Google Trends Data/2008/2008-0039-USA/alabama_2008-0039-USA.csv", skiprows = 2)
t


,Week,tornado: (Alabama)
0,2007-12-30,0
1,2008-01-06,51
2,2008-01-13,0
3,2008-01-20,0
4,2008-01-27,15
5,2008-02-03,85
6,2008-02-10,33
7,2008-02-17,100
8,2008-02-24,20
9,2008-03-02,27


In [119]:
pre2013_filepath = "/Users/eclin/Desktop/MA Thesis/02 MA Thesis Current Project/EnvData/Google Trends Data/"

In [300]:
s = "explosion: (West)"
re.findall(r"(?<=: \().*(?=\))", s)

['West']

In [325]:
pre2013_years = range(2008, 2013, 1)

pre2013_trends_dict = {}
for year in pre2013_years:
    main_filepath = pre2013_filepath + str(year)
    file_folders = [f.path for f in os.scandir(main_filepath) if f.is_dir()]

    events_in_year = {}
    for event_folder_path in file_folders:
        event_name = re.findall(r"(?<=/\d{4}/).*", event_folder_path)[0]
        
        all_files = [f for f in os.listdir(event_folder_path) if os.path.isfile(os.path.join(event_folder_path, f))]

        files_in_event = {}
        for trend_file_name in all_files:
            if trend_file_name != ".DS_Store":
                filecsv = pd.read_csv(f"{event_folder_path}/{trend_file_name}", skiprows = 2)

                col_to_strip = filecsv.columns[1]
                print(col_to_strip)
                env_topic = re.findall(r"\w+(?=:)", col_to_strip)[0]
                state = re.findall(r"(?<=: \().*(?=\))", col_to_strip)[0]
                
                filecsv["Week"] = pd.to_datetime(filecsv["Week"])
                
                files_in_event[state] = filecsv
                
        events_in_year[event_name] = files_in_event
    pre2013_trends_dict[year] = events_in_year

blizzard: (Massachusetts)
blizzard: (New York)
blizzard: (Maine)
blizzard: (New Hampshire)
explosion: (Georgia)
forest fire: (California)
wildfire: (California)
cyclone: (Illinois)
cyclone: (Texas)
cyclone: (Missouri)
cyclone: (Michigan)
cyclone: (Louisiana)
cyclone: (Tennessee)
cyclone: (Arkansas)
cyclone: (Ohio)
cyclone: (Kentucky)
cyclone: (Pennsylvania)
cyclone: (Indiana)
cyclone: (Louisiana)
cyclone: (Mississippi)
cyclone: (Texas)
cyclone: (Alabama)
city fire: (Pennsylvania)
tornado: (Georgia)
tornado: (Virginia)
tornado: (Texas)
tornado: (Maryland)
tornado: (North Carolina)
tornado: (Kansas)
tornado: (Oklahoma)
tornado: (Missouri)
tornado: (Arkansas)
air quality: (Utah)
flood: (Nebraska)
flood: (Illinois)
flood: (Wisconsin)
flood: (Iowa)
flood: (Kansas)
flood: (Michigan)
flood: (Minnesota)
flood: (Missouri)
tornado: (Illinois)
tornado: (Ohio)
tornado: (Missouri)
tornado: (Mississippi)
tornado: (Kentucky)
tornado: (Indiana)
tornado: (Arkansas)
tornado: (Alabama)
tornado: (Texas)
t

/var/folders/mf/zhnnyffn3wv77d_kx1l8l8hw0000gn/T/ipykernel_54119/1560487324.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  filecsv["Week"] = pd.to_datetime(filecsv["Week"])


storm: (North Dakota)
storm: (Ohio)
storm: (Florida)
storm: (Wyoming)
storm: (Minnesota)
storm: (Tennessee)
storm: (Maryland)
blizzard: (New York)
blizzard: (New Jersey)
blizzard: (Connecticut)
blizzard: (Massachusetts)
flood: (Montana)
flood: (North Dakota)
flood: (South Dakota)
flood: (Iowa)
flood: (Missouri)
cyclone: (District of Columbia)
cyclone: (Louisiana)
cyclone: (Georgia)
cyclone: (Virginia)
cyclone: (Mississippi)
cyclone: (Alabama)
cyclone: (Texas)
cyclone: (New York)
cyclone: (Maryland)
cyclone: (Tennessee)
cyclone: (Pennsylvania)
road conditions: (New York)
wildfire: (Colorado)
blizzard: (Pennsylvania)
blizzard: (Texas)
blizzard: (Virginia)
blizzard: (Louisiana)
blizzard: (Alabama)
blizzard: (Mississippi)
blizzard: (Arkansas)
blizzard: (New York)
blizzard: (New Jersey)
blizzard: (Indiana)
blizzard: (Wisconsin)
blizzard: (Iowa)
blizzard: (Nebraska)
blizzard: (Kansas)
tornado: (Nebraska)
tornado: (Kansas)
tornado: (Missouri)
tornado: (Illinois)
tornado: (Ohio)
tornado: (Alab

In [182]:
s = "/Users/eclin/Desktop/MA Thesis/02 MA Thesis Current Project/EnvData/Google Trends 2013-2024/2013/Lightning:Thunderstorms"



['Lightning:Thunderstorms']

In [324]:
post2013_filepath = "/Users/eclin/Desktop/MA Thesis/02 MA Thesis Current Project/EnvData/Google Trends 2013-2024/"

post2013_years = range(2013, 2025, 1)

post2013_trends_dict = {}
for year in post2013_years:
    main_filepath = post2013_filepath + str(year)
    
    file_folders = [f.path for f in os.scandir(main_filepath) if f.is_dir()]
    
    events_in_year = {}
    for file_folder in file_folders:
        #print(file_folder)
        topic = re.findall(r"(?<=/\d{4}/).*", file_folder)[0]
        topic = topic.replace(":", "/")
        
        #print(topic)
        all_files = [f for f in os.listdir(file_folder) if os.path.isfile(os.path.join(file_folder, f))]
        
        files_in_topic = {}

        for file_name in all_files:
            if file_name != ".DS_Store":
                filecsv = pd.read_csv(f"{file_folder}/{file_name}", skiprows = 2)
                
                col_to_strip = filecsv.columns[1]  
                env_topic = re.findall(r"\w+(?=:)", col_to_strip)[0]
                state = re.findall(r"(?<=: \().*(?=\))", col_to_strip)[0]

                filecsv["Week"] = pd.to_datetime(filecsv["Week"])

                files_in_topic[state] = filecsv
         
        events_in_year[topic] = files_in_event

    post2013_trends_dict[year] = events_in_year

In [273]:
envdf = pd.read_csv("/Users/eclin/Desktop/MA Thesis/02 MA Thesis Current Project/EnvData/envdata_for_analysis.csv", converters={"NewLocs": literal_eval})

In [272]:
envdf["ExpandedLocs"] = ""
for i, row in envdf.iterrows():
    loc_list = row["NewLocs"]
    expanded_locs = []
    for loc in loc_list:
        expanded_loc = state_dict[loc]
        expanded_locs.append(expanded_loc)

    envdf.at[i, "ExpandedLocs"] = expanded_locs

In [209]:
envdf_expanded = envdf.explode('ExpandedLocs')


In [274]:
df = pd.read_csv("/Users/eclin/Desktop/MA Thesis/02 MA Thesis Current Project/Data For Analysis/FINAL_ALLBILLS_DF_MERGED.csv")

/var/folders/mf/zhnnyffn3wv77d_kx1l8l8hw0000gn/T/ipykernel_54119/2505811808.py:1: DtypeWarning: Columns (16,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301

In [275]:
df = df.drop(["Unnamed: 0", "X"], axis = 1)

In [319]:
from datetime import datetime, timedelta


In [342]:
#create list of neighboring states for proximity sorting
neighbors = {
    # https://state.1keydata.com/bordering-states-list.php
    "AK": [],
    "AL": ["FL", "GA", "MS", "TN"],
    "AR": ["LA", "MS", "MO", "OK", "TN", "TX"],
    "AZ": ["CA", "CO", "NV", "NM", "UT"],
    "CA": ["AZ", "NV", "CO"],
    "CO": ["AZ", "KS", "NE", "NM", "OK", "UT", "WY"],
    "CT": ["MA", "NY", "RI"],
    "DE": ["MD", "NJ", "PA"],
    "FL": ["AL", "GA"],
    "GA": ["AL", "FL", "NC", "SC", "TN"],
    "HI": [],
    "IA": ["IL", "MN", "MO", "NE", "SD", "WI"],
    "ID": ["MT", "NV", "OR", "UT", "WA", "WY"],
    "IL": ["IN", "IA", "MI", "KY", "MO", "WI"],
    "IN": ["IL", "KY", "MI", "OH"],
    "KS": ["CO", "MO", "NE", "OK"],
    "KY": ["IL", "IN", "MO", "OH", "TN", "VA", "WV"],
    "LA": ["AR", "MS", "TX"],
    "MA": ["CT", "NH", "NY", "RI", "VT"],
    "MD": ["DE", "PA", "VA", "WV"],
    "ME": ["NH"],
    "MI": ["IL", "IN", "MN", "OH", "WI"],
    "MN": ["IA", "MI", "ND", "SD", "WI"],
    "MO": ["AR", "IL", "IA", "KS", "KY", "NE", "OK", "TN"],
    "MS": ["AL", "AK", "LA", "TN"],
    "MT": ["ID", "ND", "SD", "WY"],
    "NC": ["GA", "SC", "TN", "VA"],
    "ND": ["MN", "MT", "SD"],
    "NE": ["CO", "IA", "KS", "MO", "SD", "WY"],
    "NH": ["ME", "MA", "VT"],
    "NJ": ["DE", "NY", "PA"],
    "NM": ["AZ", "CO", "OK", "TX", "UT"],
    "NV": ["AZ", "CA", "ID", "OR", "UT"],
    "NY": ["CT", "MA", "NJ", "PA", "RI", "VT"],
    "OH": ["IN", "KY", "MI", "PA", "WV"],
    "OK": ["AR", "CO", "KS", "MO", "NM", "TX"],
    "OR": ["CA", "ID", "NV", "WA"],
    "PA": ["DE", "MD", "NJ", "NY", "OH", "WV"],
    "RI": ["CT", "MA", "NY"],
    "SC": ["GA", "NC"],
    "SD": ["IA", "MN", "MT", "NE", "ND", "WY"],
    "TN": ["AL", "AR", "GA", "KY", "MS", "MO", "NC", "VA"],
    "TX": ["AR", "LA", "NM", "OK"],
    "UT": ["AZ", "CO", "ID", "NV", "NM", "WY"],
    "VA": ["KY", "MD", "OH", "PA", "VA"],
    "VT": ["MA", "NH", "NY"],
    "WA": ["ID", "OR"],
    "WI": ["IL", "IA", "MI", "MN"],
    "WV": ["KY", "MA", "OH", "PA", "VA"],
    "WY": ["CO", "ID", "MT", "NE", "SD", "UT"],
    # https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States#Federal_district.
    "DC": ["MD", "VA"],
    "PR" : []
}

0                Tornado
1                Tornado
2                Tornado
3        Ground movement
4        Ground movement
              ...       
16568          Heat wave
16569          Heat wave
16570          Heat wave
16571     Severe weather
16572     Severe weather
Name: Disaster.Subtype, Length: 16573, dtype: object

{2013: {'Lightning/Thunderstorms': {'Oregon':           Week  blizzard: (Oregon)
   0   2012-01-01                  11
   1   2012-01-08                  11
   2   2012-01-15                  13
   3   2012-01-22                  12
   4   2012-01-29                  12
   5   2012-02-05                  10
   6   2012-02-12                  13
   7   2012-02-19                  12
   8   2012-02-26                  15
   9   2012-03-04                  13
   10  2012-03-11                  15
   11  2012-03-18                  15
   12  2012-03-25                  12
   13  2012-04-01                  13
   14  2012-04-08                  10
   15  2012-04-15                  16
   16  2012-04-22                  11
   17  2012-04-29                  15
   18  2012-05-06                  15
   19  2012-05-13                  44
   20  2012-05-20                  19
   21  2012-05-27                  17
   22  2012-06-03                  17
   23  2012-06-10                  17
   24  

In [314]:
df["start_date"] =  pd.to_datetime(df['start_date'])
df["end_date"] =  pd.to_datetime(df['start_date'])

In [385]:
# pre2013_trends_dict
# post2013_trends_dict
idx = 0
for i, row in df.iterrows():
    start_year = row["start_year"]
    state_df = row["state"]
    state = state_dict[state_df]
    print(f"working on {i}")

    if start_year < 2013: 
        env_event = row["DisNo_BE"]
        print(env_event)
        try:
            matched_trend_event = pre2013_trends_dict[start_year][env_event]
            matched_state = matched_trend_event[state]
            prev7 = row["start_date"] - timedelta(days=7)
            post7 = row["end_date"] + timedelta(days=7)

            rel_time_period = matched_state[(matched_state["Week"] >= prev7) & (matched_state["Week"] <= post7)]
            search_val = float(rel_time_period.iloc[:, 1].mean())

            
        except KeyError as e:
            print("Own State Not Found")
            neighbor_list = neighbors[state_df]
            agg_search = []
                
            for nb in neighbor_list:
                full_nb = state_dict[nb]
                try:
                    #secondtry
                    matched_state = matched_trend_event[full_nb]
                    prev7 = row["start_date"] - timedelta(days=7)
                    post7 = row["end_date"] + timedelta(days=7)
                    rel_time_period = matched_state[(matched_state["Week"] >= prev7) & (matched_state["Week"] <= post7)]
                    search_val = float(rel_time_period.iloc[:, 1].mean())

                    agg_search.append(search_val)
                except KeyError as e: 
                    print(f"Neighbor {nb} not found")
                    agg_search.append(0)
                    
            search_val = pd.Series(agg_search).mean()
        finally:
            print(search_val)
            df.at[i, "search_val"] = search_val

    else:
        distype = row["Disaster.Subtype"]
        try:
            matched_type = post2013_trends_dict[distype][state]
            prev7 = row["start_date"] - timedelta(days=7)
            post7 = row["end_date"] + timedelta(days=7)

            rel_time_period = matched_state[(matched_state["Week"] >= prev7) & (matched_state["Week"] <= post7)]
            search_val = float(rel_time_period.iloc[:, 1].mean())
        except:
            print("Own State Not Found")
            neighbor_list = neighbors[state_df]
            agg_search = []
                
            for nb in neighbor_list:
                full_nb = state_dict[nb]
                if len(full_nb) != 0:
                    try:
                        #secondtry
                        matched_state = post2013_trends_dict[distype][full_nb]
                        prev7 = row["start_date"] - timedelta(days=7)
                        post7 = row["end_date"] + timedelta(days=7)
                        rel_time_period = matched_state[(matched_state["Week"] >= prev7) & (matched_state["Week"] <= post7)]
                        search_val = float(rel_time_period.iloc[:, 1].mean())
    
                        agg_search.append(search_val)
                    except KeyError as e: 
                        print(f"Neighbor {nb} not found")
                        agg_search.append(0)
                else:
                    agg_search.append(0)
        
            search_val = pd.Series(agg_search).mean()
        finally:   
            print(search_val)
            df.at[i, "search_val"] = search_val
        

working on 0
2008-0039-USA
Own State Not Found
Neighbor MD not found
Neighbor PA not found
Neighbor VA not found
19.3
working on 1
2008-0039-USA
Own State Not Found
Neighbor MD not found
Neighbor PA not found
Neighbor VA not found
19.3
working on 2
2008-0039-USA
Own State Not Found
Neighbor MD not found
Neighbor PA not found
Neighbor VA not found
19.3
working on 3
2008-0099-USA
Own State Not Found
Neighbor AZ not found
Neighbor CO not found
7.5
working on 4
2008-0099-USA
Own State Not Found
Neighbor AZ not found
Neighbor CO not found
7.5
working on 5
2008-0099-USA
Own State Not Found
Neighbor AZ not found
Neighbor CO not found
7.5
working on 6
2008-0099-USA
Own State Not Found
Neighbor CA not found
Neighbor ID not found
Neighbor WA not found
5.625
working on 7
2008-0099-USA
Own State Not Found
Neighbor AZ not found
Neighbor CO not found
7.5
working on 8
2008-0099-USA
Own State Not Found
Neighbor AZ not found
Neighbor CO not found
7.5
working on 9
2008-0143-USA
Own State Not Found
Neigh

In [387]:
df["search_val_filled"] = np.where(df["search_val"].isna(), 0, df["search_val"])

In [399]:
to_export = df[["bill_num", "DisNo_BE", "year", "search_val_filled"]]
to_export.to_csv("/Users/eclin/Desktop/MA Thesis/02 MA Thesis Current Project/Data For Analysis/df_search_to_merge.csv", index = False)